# WizBee Playground

## Searching

Search database for similar objects and ask OpenAI to generate a response.

### Load libraries and environment variables

In [15]:
from dotenv import load_dotenv
from os import getenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain


load_dotenv()

API = getenv('OPENAI_API_KEY')
REDIS_PASSWORD = getenv('REDIS_PASSWORD')
REDIS_URL = getenv('REDIS_URL')

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(openai_api_key=API, model=EMBEDDING_MODEL)
llm = OpenAI(temperature=0.1, openai_api_key=API)


### Content Redis

Connect to Redis database and load content.

In [7]:
vector_db = Redis.from_existing_index(embeddings, redis_url="redis://:" + REDIS_PASSWORD + "@"+ REDIS_URL, index_name='link')

### Create Template for OpenAI Chat

Here you can improve can do prompt engineering.

In [12]:
template = """You are a helpful bot which answers user question about b.tree a beekeeping software! Please answer the question based on the given context. The context could be from multiple references  which are separated by ;;; Answer the question using only that information. If you can't create a good answer with the references, say "Sorry, we don't have any answers in our references stored for your question.".

Question: {query}

Context: {context}
"""

prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template=template
)

### Search Redis for a document

In [17]:
query = "Why should I use b.tree?"
results = vector_db.similarity_search(query, k=1)

context = []
metadata = []
for result in results:
    context.append(result.page_content)
    metadata.append(result.metadata)

print(context)

['title: b.tree First Steps\ndescription: "Documentation and Help for Beekeeping Software and Application"\nlayout: "../../layouts/IndexLayout.astro"\nlang: "en"\n\nWhy should I use this app?\n\nThis web application was designed to help you keep track of the recent and past beekeeping tasks of your colonies. The basic idea was an easy record keeping tool, which is mandatory for european beekeepers when applying medications and to help fulfil the notation conditions (feeding, harvesting) for certified organic beekeepers. In addition, the collected data can be used as basis for statistical analysis of the beekeeping operation efficiency. As an example, one can compare the honey yield of apiaries or judge queens on their performance ratings.']


### Ask OpenAI to generate a response

In [18]:
context = ";;;".join(context)
question_chain = LLMChain(llm=llm, prompt=prompt_template)
question_chain.run(query=query, context=context)


';;;\n\nb.tree is a great tool for beekeepers to help them keep track of their beekeeping tasks, fulfill notation conditions, and analyze their beekeeping operation efficiency. It provides an easy record keeping system, which is mandatory for European beekeepers when applying medications, and can be used to compare honey yield of apiaries or judge queens on their performance ratings. With b.tree, beekeepers can easily manage their colonies and get the most out of their beekeeping operations.'